In [ ]:
# Importing Google Drive module from Google Colab
from google.colab import drive

# Mounting Google Drive to access its contents in the Colab environment
drive.mount('/content/drive')


In [ ]:
!pip install -r "requirement.txt" -q

In [ ]:
%%writefile model.py
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig
from trl import SFTTrainer
from huggingface_hub import login
import dotenv

class Train:
    """
    A class to handle the training of a LLaMA model for a specific task.

    Attributes:
    ----------
    data : str
        The dataset identifier from Hugging Face hub.
    train_size : str
        The portion of the dataset to be used for training.
    model_id : str
        The identifier of the pre-trained model to be fine-tuned.
    new_model : str
        The directory to save the newly trained model.
    """

    def __init__(self):
        self.data = "b-mc2/sql-create-context"
        self.train_size = "train[:10]"  # Picking first 10 rows for faster training
        self.model_id = "meta-llama/Llama-2-7b-chat-hf"
        self.new_model = "./llama-2-7b-chat-v2"

    def combine_fields(self, example):
        """
        Combines multiple fields from the dataset into a single 'text' field.

        Parameters:
        ----------
        example : dict
            A dictionary containing the dataset fields.

        Returns:
        -------
        dict
            A dictionary with a combined 'text' field.
        """
        example['text'] = f"Context: {example['context']} Question: {example['question']} Answer: {example['answer']}"
        return example

    def train_llama(self):
        """
        Trains a LLaMA model using the specified dataset and configurations.
        """
        # Load environment variables from .env file
        dotenv.load_dotenv('./Variables.env')

        # Login to Hugging Face Hub using the API key from environment variables
        login(token=os.environ.get('hf_key'))

        # Load and preprocess the dataset
        dataset = load_dataset(self.data, split=self.train_size)
        dataset = dataset.map(self.combine_fields)
        dataset = dataset.remove_columns(['answer', 'question', 'context'])

        # Define model configurations
        base_model_id = self.model_id
        compute_dtype = getattr(torch, "float16")

        quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=compute_dtype,
            bnb_4bit_use_double_quant=False,
        )

        # Load pre-trained model with quantization
        model = AutoModelForCausalLM.from_pretrained(
            base_model_id,
            quantization_config=quant_config,
            device_map={"": 0}
        )
        model.config.use_cache = False
        model.config.pretraining_tp = 1

        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "right"

        # Define PEFT (Parameter-Efficient Fine-Tuning) parameters
        peft_params = LoraConfig(
            lora_alpha=16,
            lora_dropout=0.1,
            r=64,
            bias="none",
            task_type="CAUSAL_LM"
        )

        # Define training parameters
        training_params = TrainingArguments(
            output_dir="./results",
            num_train_epochs=1,
            per_device_train_batch_size=4,
            gradient_accumulation_steps=8,
            optim="paged_adamw_32bit",
            save_steps=25,
            logging_steps=25,
            learning_rate=2e-4,
            weight_decay=0.001,
            fp16=True,  # Enable mixed precision training
            bf16=False,
            max_grad_norm=0.3,
            max_steps=-1,
            warmup_ratio=0.03,
            group_by_length=True,
            lr_scheduler_type="constant",
            report_to="tensorboard"
        )

        # Initialize the trainer
        trainer = SFTTrainer(
            model=model,
            train_dataset=dataset,
            peft_config=peft_params,
            dataset_text_field="text",
            max_seq_length=None,
            tokenizer=tokenizer,
            args=training_params,
            packing=False,
        )

        # Train the model
        trainer.train()

        # Save the trained model
        trainer.save_model(self.new_model)


In [ ]:
import model
modelsample = model.Train()
modelsample.train_llama()

In [ ]:
%%writefile app.py
import streamlit as st
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv
from langchain import PromptTemplate
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline
)
from huggingface_hub import login
import pandas as pd
from sqlalchemy import create_engine, text
import logging
import re
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import os
import dotenv

# Load environment variables from .env file
dotenv.load_dotenv('./.env')

# Function to create a database connection
def create_connection():
    user = os.environ.get('user')
    password = os.environ.get('password')
    host = os.environ.get('host')
    port = os.environ.get('port')  # Default port for PostgreSQL is 5432
    database = os.environ.get('database')

    connection_string = f'postgresql://{user}:{password}@{host}:{port}/{database}'
    logging.debug(f'Connecting to database with connection string: {connection_string}')
    engine = create_engine(connection_string)
    return engine.connect()

# Function to execute a SQL query and return the result as a DataFrame
def execute_query(query):
    try:
        conn = create_connection()
        logging.debug(f'Executing query: {query}')
        df = pd.read_sql_query(text(query), conn)
        conn.close()
        return df
    except Exception as e:
        logging.error(f'Error executing query: {e}')
        return None

# Streamlit page configuration
st.set_page_config(page_title="Streamlit SQL Chatbot", page_icon="🤖")
st.title("SQL Chatbot")

# Function to generate a SQL query based on user input
def get_response(user_query):
    login(token=os.environ.get('hf_key'))

    prompt = user_query
    print("Prompt:", prompt)
    model_name = "./llama-2-7b-chat-v2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

    result = pipe(f"<s>[INST]<<sys>>I want you to act as a SQL query expert. Convert this sentence: {prompt} : into PostgreSQL query <</sys>>[/INST]")
    generated_text = result[0]['generated_text']

    sql_query = extract_sql_query(generated_text)
    print("Generated Text:", generated_text)
    print("SQL Query:", sql_query)
    return sql_query

# Function to extract SQL query from generated text
def extract_sql_query(input_text):
    sql_query_match = re.search(r'```(?:sql\n)?(.*?)\n```', input_text, re.DOTALL)
    if sql_query_match:
        sql_query = sql_query_match.group(1).strip()
    else:
        sql_query = "Unable to extract SQL query."
    return sql_query

# Initialize session state for chat history and DataFrames
if 'chat_history' not in st.session_state:
    st.session_state.chat_history = [AIMessage(content="Hello, I am a bot. How can I help you?")]
if 'dataframes' not in st.session_state:
    st.session_state.dataframes = [None]  # Initialize with None for the initial bot message

# Display chat history
for idx, message in enumerate(st.session_state.chat_history):
    if isinstance(message, AIMessage):
        with st.chat_message("AI"):
            st.write(message.content)
            if st.session_state.dataframes[idx] is not None:
                st.dataframe(st.session_state.dataframes[idx])
    elif isinstance(message, HumanMessage):
        with st.chat_message("Human"):
            st.write(message.content)

# User input
user_query = st.chat_input("Type your message here...")
if user_query is not None and user_query != "":
    st.session_state.chat_history.append(HumanMessage(content=user_query))

    with st.chat_message("Human"):
        st.markdown(user_query)

    with st.chat_message("AI"):
        response = get_response(user_query)
        df = execute_query(response)
        st.write(response)
        st.dataframe(df)

        # Store the response and DataFrame in the session state
        st.session_state.chat_history.append(AIMessage(content=response))
        st.session_state.dataframes.append(None)
        st.session_state.dataframes.append(df)


In [ ]:
# Note: Replace [authtoken] with your actual ngrok authtoken
!ngrok authtoken [authtoken]

# Start ngrok tunnel on port 8501 (default for Streamlit)
public_url = ngrok.connect(8501)

# Print the public URL generated by ngrok
print('Public URL:', public_url)

# Run the Streamlit app in the background
# Note: 'app.py' should be your Streamlit application script
!streamlit run app.py &>/dev/null&